<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Get an organization
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Get+an+organization:+Error+short+description">Bug report</a>

**Tags:** #github #organization #automation #snippet

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-06-25 (Created: 2024-06-18)

**Description:** This notebook allows users to retrieve organization data from a repository.

## Input

### Import libraries

In [1]:
import requests
import re
import pandas as pd
import naas

### Setup variables

- `github_token`: personal token
- `repo_url`: link to the chosen github repo
- `company_name`: name of the company
- `api_key`: token to get linkedin
- `output_csv`: excel file

In [2]:
github_token = naas.secret.get("GITHUB_TOKEN")
repo_url = "https://github.com/jupyter-naas/awesome-notebooks"
company_name = "naas.ai"
api_key = "899fa50c3a6c56e8b19e37608140f327bf9bf807"
output_csv = f"{repo_url.split('github.com/')[1].replace('/', '_')}_organization.csv"

## Model


### Get github details

In [3]:
def get_repo_details(repo_url, token):
    # Extract the repo path from the URL
    repo_path = repo_url.replace("https://github.com/", "")
    
    # GitHub API endpoint for repository details
    api_url = f"https://api.github.com/repos/{repo_path}"
    
    # Headers with the authorization token
    headers = {
        "Authorization": f"token {token}"
    }
    
    # Send GET request to the GitHub API
    response = requests.get(api_url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        repo_details = response.json()
        return repo_details
    else:
        return f"Error: {response.status_code} - {response.json()['message']}"
repo_details = get_repo_details(repo_url, github_token)
    
def get_company_details(domain, api_key):
    # Hunter.io API endpoint for company information
    url = f"https://api.hunter.io/v2/domain-search?domain={domain}&api_key={api_key}"
    
    # Send GET request to the Hunter.io API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        if 'data' in data:
            return data['data']
        else:
            return "Company details not found"
    else:
        return f"Error: {response.status_code} - {response.json().get('errors', 'Unknown error')}"
    
linkedin_details = get_company_details(company_name, api_key)

## Output


### Display result

In [4]:
data = []

data.append({
    "URL": repo_details["url"],
    "NAME": repo_details["name"],
    "ID": repo_details["id"],
    "OWNER_LOGIN": repo_details["owner"]["login"],
    "OWNER_ID": repo_details["owner"]["id"],
    "DESCRIPTION": repo_details["description"],
    "CREATED_AT": repo_details["created_at"],
    "TOPICS": repo_details["topics"],
    "NUMBER_OF_OPEN_ISSUES": repo_details["open_issues"],
    "LINKEDIN_URL": linkedin_details["linkedin"],
    "LINKEDIN_DESCRIPTION": linkedin_details["description"],
    "INDUSTRY": linkedin_details["industry"],
    "TECHNOLOGIES": linkedin_details["technologies"],
    "CITY": linkedin_details["city"],
    "HEADCOUNT": linkedin_details["headcount"],
    "COMPANY_TYPE": linkedin_details["company_type"]
})

df = pd.DataFrame(data)
df

### Save dataframe to csv

In [5]:
df.to_csv(output_csv, index=False)